# Secure Data Disclosure on Kubernetes: Deployment and Server Administration

This notebook showcases how a data owner could set up the service on a kubernetes cluster, add and make their data available to certain user. We will do this in a step by step fashion.

## Deploying the service

### Building the server image
The SDD service is comprised of a fastapi server and a MongoDB database for keeping state and administration. While the database image is public, the server image must first be built and pushed to a registry.

NOTE: For now, the server configuration file is copied and put into the server container. This is of course not practical (and not safe, since the configuration file contains passwords and secrets) and will be updated in future versions. The `config/example_config.yaml` is the one that is copied into the container. One has to change it and rebuild+push the server container in order to change the server configuration.

In [ ]:
# !docker login (=> use personal token from dockerhub, has to be done only once)

!cd .. && docker build --target sdd_server_prod -t <your_registry>/sdd_server_prod:latest .
!cd .. && docker push <your_registry>/sdd-poc-server:latest

### Deploying the service Helm chart
We use a Helm chart to deploy the service on a Kubernetes cluster. The sdd-server chart is located at `deploy/helm/charts/sdd_server`, let us change our working directory to this location.

In [1]:
import os
os.chdir('../deploy/helm/charts/sdd_server')

The `values.yaml` file contains all the configuration values for the service. We must now update the `image.repository` field to the one we pushed the server container image to. One can also change the url to which the service will be published with `ingress.hosts[0].host` (or disable this feature by setting `ingress.enabled` to `False`).

    => Update `values.yaml` file

As previously stated, the service is made up of a server and a MongoDB database. Before installing the chart, we must thus first download that dependency.

In [2]:
!helm dependency update

Saving 1 charts
Pulled: registry-1.docker.io/bitnamicharts/mongodb:13.18.1
Digest: sha256:f3b2a691537260044746bc4a8898e9ae68e8c29864639737b6da920f99aebe97
Deleting outdated charts


Now the chart is ready to be installed, so let the magic happen!

In [ ]:
!helm install -f values.yaml sdd-service .

The installation notes show the url at which the server is exposed. One can have a look at the api docummentation by visiting `<server_url>/docs`

One can also check the whether the service started error free by using the `kubectl get all` command as well as inspecting the server logs with `kubectl logs <server-pod-name>`

## Administering the service by accessing the mongoDB

Let's switch directory again and move to the administration script.

In [5]:
import os
os.chdir('../../../../src/')

To interact with the mongoDB, we will need to install a few libraries. Let's do so by creating a python virtual environment and installing the dependencies listed in `admin_requirements.txt`

In [9]:
!pip install -r ../admin_requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 8.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 9.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 9.3 MB/s eta 0:00:00ta 

We should now have the required environment to interact with the admin database.

### Preparing the database

You can visualise all the options offered by the database by running the command `python mongodb_admin.py --help`. We will go through through each of them in the rest of the notebook.

In [12]:
!python mongodb_admin.py --help

usage: MongoDB administration script for the user database [-h]
                                                           {add_user,add_user_with_budget,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,show_user,create_users_collection,add_dataset,add_datasets,drop_collection,show_collection}
                                                           ...

options:
  -h, --help            show this help message and exit

subcommands:
  {add_user,add_user_with_budget,del_user,add_dataset_to_user,del_dataset_to_user,set_budget_field,set_may_query,show_user,create_users_collection,add_dataset,add_datasets,drop_collection,show_collection}
                        user database administration operations
    add_user            add user to users collection
    add_user_with_budget
                        add user with budget to users collection
    del_user            delete user from users collection
    add_dataset_to_user
                        add dataset w

Let's first make sure the database is empty and in a clean state.

In [30]:
!python mongodb_admin.py drop_collection --collection datasets
!python mongodb_admin.py drop_collection --collection metadata
!python mongodb_admin.py drop_collection --collection users

Deleted collection datasets.
Deleted collection metadata.
Deleted collection users.


### Datasets (add and drop)

We first need to set the dataset meta-information. For each dataset, 2 informations are required:
- the type of database in which the dataset is stored
- a path to the metadata of the dataset (stored as a yaml file).

To later perform query on the dataset, metadata are required. In this secure server the metadata information is expected to be in the same format as [SmartnoiseSQL dictionary format](https://docs.smartnoise.org/sql/metadata.html#dictionary-format), where among other, there is information about all the available columns, their type, bound values (see Smartnoise page for more details). It is also expected to be in a `yaml` file.

These information (dataset name, dataset type and metadata path) are stored in the `datasets` collection. Then for each dataset, its metadata is fetched from its `yaml` file and stored in a collection named `metadata`.

We then check that there is indeed no data in the dataset and metadata collections yet:

In [27]:
!python mongodb_admin.py show_collection --collection datasets

[]


In [28]:
!python mongodb_admin.py show_collection --collection metadata

[]


We can add **one dataset** with its name, database type and path to medata file:

In [34]:
os.getcwd()

'/home/onyxia/work/sdd-poc-server/server/src'

In [35]:
!python mongodb_admin.py add_dataset -d PENGUIN -db CONSTANT_PATH_DB -mp ../data/collections/metadata/penguin_metadata.yaml

Added dataset PENGUIN with database CONSTANT_PATH_DB and metadata from ../data/collections/metadata/penguin_metadata.yaml.


We can now see the dataset and metadata collection with the Iris dataset:

In [36]:
!python mongodb_admin.py show_collection --collection datasets

[{'dataset_name': 'PENGUIN', 'database_type': 'CONSTANT_PATH_DB', 'metadata_path': '../data/collections/metadata/penguin_metadata.yaml'}]


In [37]:
!python mongodb_admin.py show_collection --collection metadata

[{'PENGUIN': {'': {'Schema': {'Table': {'max_ids': 1, 'row_privacy': True, 'censor_dims': False, 'species': {'type': 'string', 'cardinality': 3, 'categories': ['Adelie', 'Chinstrap', 'Gentoo']}, 'island': {'type': 'string', 'cardinality': 3, 'categories': ['Torgersen', 'Biscoe', 'Dream']}, 'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0}, 'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0}, 'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0}, 'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0}, 'sex': {'type': 'string', 'cardinality': 2, 'categories': ['MALE', 'FEMALE']}}}}, 'engine': 'csv'}}]


Or a path to a yaml file which contains all these informations to do **multiple datasets** in one command:

In [39]:
!python mongodb_admin.py add_datasets --path ../data/collections/dataset_collection.yaml -c

Cleaning done. 
Added datasets collection from yaml at ../data/collections/dataset_collection.yaml. 
Added metadata of IRIS dataset. 
Added metadata of PENGUIN dataset. 
Added metadata of TITANIC dataset. 
Added metadata of FSO_INCOME_SYNTHETIC dataset. 


The argument *-c* or *--clean* allows you to clear the current dataset collection before adding your collection.

By default, *add_datasets* will only add new dataset found from the collection provided.

In [40]:
!python mongodb_admin.py add_datasets --path ../data/collections/dataset_collection.yaml

Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 


Arguments :

*-od* / *--overwrite_datasets* : Overwrite the values for **exisiting datasets** with the values provided in the yaml.

*-om* / *--overwrite_metadata* : Overwrite the values for **exisiting metadata** with the values provided in the yaml.

In [41]:
# Add new datasets/metadata, update existing datasets
!python mongodb_admin.py add_datasets --path ../data/collections/dataset_collection.yaml -od

Existing datasets updated with values from yaml at ../data/collections/dataset_collection.yaml. 
Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 
Metadata already exist. User the command -om to overwrite with new values. 


In [42]:
# Add new datasets/metadata, update existing metadata
!python mongodb_admin.py add_datasets --path ../data/collections/dataset_collection.yaml -om

Metadata updated with values from yaml for dataset : IRIS. 
Metadata updated with values from yaml for dataset : PENGUIN. 
Metadata updated with values from yaml for dataset : TITANIC. 
Metadata updated with values from yaml for dataset : FSO_INCOME_SYNTHETIC. 


In [43]:
# Add new datasets/metadata, update existing datasets & metadata
!python mongodb_admin.py add_datasets --path ../data/collections/dataset_collection.yaml -od -om

Existing datasets updated with values from yaml at ../data/collections/dataset_collection.yaml. 
Metadata updated with values from yaml for dataset : IRIS. 
Metadata updated with values from yaml for dataset : PENGUIN. 
Metadata updated with values from yaml for dataset : TITANIC. 
Metadata updated with values from yaml for dataset : FSO_INCOME_SYNTHETIC. 


Let's see all the dataset collection:

In [44]:
!python mongodb_admin.py show_collection --collection datasets

[{'dataset_name': 'IRIS', 'database_type': 'REMOTE_HTTP_DB', 'dataset_url': 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv', 'metadata_path': '../data/collections/metadata/iris_metadata.yaml'}, {'dataset_name': 'PENGUIN', 'database_type': 'REMOTE_HTTP_DB', 'dataset_url': 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv', 'metadata_path': '../data/collections/metadata/penguin_metadata.yaml'}, {'dataset_name': 'TITANIC', 'database_type': 'S3_DB', 's3_bucket': 'example', 's3_key': 'data/titanic.csv', 'endpoint_url': 'https://api-sdd-minio.lab.sspcloud.fr', 'aws_access_key_id': 'admin', 'aws_secret_access_key': 'admin123', 'metadata_path': '../data/collections/metadata/titanic_metadata.yaml'}, {'dataset_name': 'FSO_INCOME_SYNTHETIC', 'database_type': 'LOCAL_DB', 'dataset_path': '../data/datasets/income_synthetic_data.csv', 'metadata_path': '../data/collections/metadata/fso_income_synthetic_metadata.yaml'}]


Finally let's have a look at the  stored metadata:

In [45]:
!python mongodb_admin.py show_collection --collection metadata

[{'IRIS': {'': {'Schema': {'Table': {'max_ids': 1, 'petal_length': {'type': 'float', 'lower': 0.5, 'upper': 10.0}, 'petal_width': {'type': 'float', 'lower': 0.05, 'upper': 5.0}, 'row_privacy': True, 'sepal_length': {'type': 'float', 'lower': 2.0, 'upper': 10.0}, 'sepal_width': {'type': 'float', 'lower': 1.0, 'upper': 6.0}, 'species': {'type': 'string', 'cardinality': 3, 'categories': ['setosa', 'versicolor', 'virginica']}}}}, 'engine': 'csv'}}, {'PENGUIN': {'': {'Schema': {'Table': {'max_ids': 1, 'row_privacy': True, 'censor_dims': False, 'species': {'type': 'string', 'cardinality': 3, 'categories': ['Adelie', 'Chinstrap', 'Gentoo']}, 'island': {'type': 'string', 'cardinality': 3, 'categories': ['Torgersen', 'Biscoe', 'Dream']}, 'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0}, 'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0}, 'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0}, 'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper

### Users

#### Adding users
Let's see which users are alreay loaded:

In [46]:
!python mongodb_admin.py show_collection --collection users

[]


And now let's add few users.

In [47]:
!python mongodb_admin.py add_user_with_budget --user 'Mrs. Daisy' --dataset 'IRIS' --epsilon 10.0 --delta 0.001

Added access to user Mrs. Daisy with dataset IRIS, budget epsilon 10.0 and delta 0.001.


In [48]:
!python mongodb_admin.py add_user_with_budget --user 'Mr. Coldheart' --dataset 'PENGUIN' --epsilon 10.0 --delta 0.001

Added access to user Mr. Coldheart with dataset PENGUIN, budget epsilon 10.0 and delta 0.001.


In [49]:
!python mongodb_admin.py add_user_with_budget --user 'Lord McFreeze' --dataset 'PENGUIN' --epsilon 10.0 --delta 0.001

Added access to user Lord McFreeze with dataset PENGUIN, budget epsilon 10.0 and delta 0.001.


Users must all have different names, otherwise you will have an error and nothing will be done:

In [50]:
!python mongodb_admin.py add_user_with_budget --user 'Lord McFreeze' --dataset 'IRIS' --epsilon 10.0 --delta 0.001

Traceback (most recent call last):
  File "/home/onyxia/work/sdd-poc-server/server/src/mongodb_admin.py", line 549, in <module>
    args.func(args)
  File "/home/onyxia/work/sdd-poc-server/server/src/mongodb_admin.py", line 47, in add_user_with_budget
    raise ValueError("Cannot add user because already exists. ")
ValueError: Cannot add user because already exists. 


If you want to add another dataset access to an existing user, just use the function `add_dataset_to_user` command.

In [51]:
!python mongodb_admin.py add_dataset_to_user --user 'Lord McFreeze' --dataset 'IRIS' --epsilon 5.0 --delta 0.005

Added access to dataset IRIS to user Lord McFreeze with budget epsilon 5.0 and delta 0.005.


Alternatively, you can create a user without assigned dataset and then add dataset in another command.

In [52]:
!python mongodb_admin.py add_user --user 'Madame Frostina'

Added user Madame Frostina.


In [53]:
!python mongodb_admin.py add_dataset_to_user --user 'Madame Frostina' --dataset 'IRIS' --epsilon 5.0 --delta 0.005

Added access to dataset IRIS to user Madame Frostina with budget epsilon 5.0 and delta 0.005.


In [54]:
!python mongodb_admin.py add_dataset_to_user --user 'Madame Frostina' --dataset 'PENGUIN' --epsilon 5.0 --delta 0.005

Added access to dataset PENGUIN to user Madame Frostina with budget epsilon 5.0 and delta 0.005.


And we can also modify existing the total budget of a user:

In [55]:
!python mongodb_admin.py add_user_with_budget --user 'Dr. Antartica' --dataset 'PENGUIN' --epsilon 10.0 --delta 0.001

Added access to user Dr. Antartica with dataset PENGUIN, budget epsilon 10.0 and delta 0.001.


In [56]:
!python mongodb_admin.py set_budget_field --user 'Dr. Antartica' --dataset 'PENGUIN' --field initial_epsilon --value 20.0

Set budget of Dr. Antartica for dataset PENGUIN of initial_epsilon to 20.0.


Let's see the current state of the database:

In [57]:
!python mongodb_admin.py show_collection --collection users

[{'user_name': 'Mrs. Daisy', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Mr. Coldheart', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Lord McFreeze', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Madame Frostina', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'PENGUIN', 'initial_epsilon': 5.0, 'initial_delta': 0.00

Do not hesitate to re-run this command after every other command to ensure that everything runs as expected.

#### Removing users
You have just heard that the penguin named Coldheart might have malicious intentions and decide to remove his access until an investigation has been carried out. To ensure that he is not allowed to do any more queries, run the following command:

In [58]:
!python mongodb_admin.py set_may_query --user 'Mr. Coldheart' --value False

Set user Mr. Coldheart may query.


Now, he won't be able to do any query (unless you re-run the query with --value True).

A few days have passed and the investigation reveals that he was aiming to do unethical research, you can remove his dataset by doing:

In [59]:
!python mongodb_admin.py del_dataset_to_user --user 'Mr. Coldheart' --dataset 'PENGUIN'

Remove access to dataset PENGUIN from user Mr. Coldheart.


Or delete him completely from the codebase:

In [60]:
!python mongodb_admin.py del_user --user 'Mr. Coldheart'

Deleted user Mr. Coldheart.


Let's see the resulting users:

In [61]:
!python mongodb_admin.py show_collection --collection users

[{'user_name': 'Mrs. Daisy', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Lord McFreeze', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Madame Frostina', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'PENGUIN', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Dr. Antartica', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 20.0, 'initial_delta': 0.00

#### Changing the budget
You also change your mind about the budget allowed to Lord McFreeze and give him a bit more on the penguin dataset.

In [62]:
!python mongodb_admin.py set_budget_field --user 'Lord McFreeze' --dataset 'PENGUIN' --field initial_epsilon --value 15.0

Set budget of Lord McFreeze for dataset PENGUIN of initial_epsilon to 15.0.


In [63]:
!python mongodb_admin.py set_budget_field --user 'Lord McFreeze' --dataset 'PENGUIN' --field initial_delta --value 0.005

Set budget of Lord McFreeze for dataset PENGUIN of initial_delta to 0.005.


Let's check all our changes by looking at the state of the database:

In [64]:
!python mongodb_admin.py show_collection --collection users

[{'user_name': 'Mrs. Daisy', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 10.0, 'initial_delta': 0.001, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Lord McFreeze', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 15.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Madame Frostina', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}, {'dataset_name': 'PENGUIN', 'initial_epsilon': 5.0, 'initial_delta': 0.005, 'total_spent_epsilon': 0.0, 'total_spent_delta': 0.0}]}, {'user_name': 'Dr. Antartica', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 20.0, 'initial_delta': 0.00

### Finally, everything can actually be loaded directly from a single file

Let's delete the existing user collection first:

In [65]:
!python mongodb_admin.py drop_collection --collection users

Deleted collection users.


Is is now empty:

In [66]:
!python mongodb_admin.py show_collection --collection users

[]


We add the data based on a yaml file:

In [68]:
!python mongodb_admin.py create_users_collection --path ../data/collections/user_collection.yaml

Added user data from yaml at ../data/collections/user_collection.yaml.


By default, *create_users_collection* will only add new users to the database.

In [69]:
!python mongodb_admin.py create_users_collection --path ../data/collections/user_collection.yaml

No new users added, they already exist in the server


If you want to clean the current users collection and replace it, you can use the argument *--clean*. 

In [70]:
!python mongodb_admin.py create_users_collection --path ../data/collections/user_collection.yaml --clean

Cleaning done. 

Added user data from yaml at ../data/collections/user_collection.yaml.


If you want to add new users and update the existing ones in your collection, you can use the argument *--overwrite*. This will make sure to add new users if they do not exist and replace values from existing users with the collection provided.

In [71]:
!python mongodb_admin.py create_users_collection --path ../data/collections/user_collection.yaml --overwrite

Existing users updated. 
No new users added, they already exist in the server


And let's see the resulting collection:

In [72]:
!python mongodb_admin.py show_collection --collection users

[{'user_name': 'Alice', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 10, 'initial_delta': 0.0001, 'total_spent_epsilon': 1, 'total_spent_delta': 1e-06}, {'dataset_name': 'PENGUIN', 'initial_epsilon': 5, 'initial_delta': 0.0005, 'total_spent_epsilon': 0.2, 'total_spent_delta': 1e-07}]}, {'user_name': 'Dr. Antartica', 'may_query': True, 'datasets_list': [{'dataset_name': 'PENGUIN', 'initial_epsilon': 45, 'initial_delta': 0.005, 'total_spent_epsilon': 0, 'total_spent_delta': 0}]}, {'user_name': 'Dr. FSO', 'may_query': True, 'datasets_list': [{'dataset_name': 'FSO_INCOME_SYNTHETIC', 'initial_epsilon': 45, 'initial_delta': 0.005, 'total_spent_epsilon': 0, 'total_spent_delta': 0}]}, {'user_name': 'Bob', 'may_query': True, 'datasets_list': [{'dataset_name': 'IRIS', 'initial_epsilon': 10, 'initial_delta': 0.0001, 'total_spent_epsilon': 0, 'total_spent_delta': 0}]}, {'user_name': 'Jack', 'may_query': True, 'datasets_list': [{'dataset_name': 'TITANIC', 'initia

## Archives of queries

In [ ]:
!python mongodb_admin show_collection --collection queries_archives

## Stopping the service: Let's not do it right now!

To tear down the service, we simply execute the command `helm uninstall sdd-service`